<a href="https://colab.research.google.com/github/SonolaMoyo/Activity1/blob/main/StageD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The stakes of this project are paramount to share knowledge on computer vision applied to forestry and generalize these techniques to other forest monitoring use-cases. Datasets and AI projects on forests are still scarced, so there is a growing need to create massive catalog and gather intelligence about biodiversity. Ultimately, it will benefits and attract more company like Pachama or Avalanche to protect and get a deeper understanding of our forests.

The quantity of labelized images in this challenge is limited (about 40,000), so we need to retrain existing models on this small Planet dataset. Resnet has shown to be a strong candidate on previous challenges among pretrained models like VGG, Inception and DenseNet (full list of pretrained torchvision models here). New deep learning techniques are emerging and ideas worth exploring include the following:

Cossim CNN 

Vision Transformers


Self-supervised learning


This dataset initially contained 4 bands satellite imagery —R, G, B and Near-Infrared (NIR)—, available through torrent download. The torrents seem to have been disabled but fortunately, a Kaggler have saved the entire RGB .png dataset. The .tif files containing the NIR dataset however, aren't available anymore. NIR is especially useful to compute greenness indexes such as the NDVI. It allows a finer visualisation of the tree canopee and measurement of the carbon absorption, although we would need to recreate a model from scratch to use it, since all models have been pretrained on RGB only.

The dataset contains a wide class imbalance, with primary forest presence on more than 90% of the labels, whereas conventional mining only account for 0.25%. We need to monitor the scores for these rare classes, as they are the key to an overall good score and an efficient model to detect ponctual human activity.

In [1]:
import os
import random
from time import time
from glob import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import dill as pickle

from plotly import graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from matplotlib import pyplot as plt
%matplotlib inline

import cv2

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T, models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
!pip install -q torchsummary --user
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cpu


We set random seeding for reproducibility purposes.

In [2]:
random.seed(101)
np.random.seed(101)
torch.manual_seed(101);